<a href="https://colab.research.google.com/github/reallygooday/60daysofudacity/blob/master/Story_Generator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>



Character-Level LSTM in PyTorch

https://classroom.udacity.com/courses/ud188/lessons/a8fc0724-37ed-40d9-a226-57175b8bb8cc/concepts/cfeeb323-45d6-4567-9a7b-49700a25dcfe

https://colab.research.google.com/drive/1jzNdctJwpG7d81Pu6-2HZPLqzCcnHypV#scrollTo=9AqrWknkGye2

https://github.com/udacity/deep-learning-v2-pytorch/blob/master/recurrent-neural-networks/char-rnn/Character_Level_RNN_Solution.ipynb

In [0]:
import numpy as np
import torch# open text file and read in data as `text`
from torch import nn
import torch.nn.functional as F

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
!ls '/content/drive/My Drive/Text Generator'

'generated_script2019-07-30 17:15:13.735175.txt'
 oliver.txt
 place2.txt
 place.txt
'preprocessed2019-07-30 17:06:00.732826.p'
'preprocessed2019-07-30 17:06:42.389736.p'
'preprocessed2019-07-30 17:14:12.698497.p'
'preprocessed2019-07-30 17:17:30.349953.p'
'trained_scriptGenModel2019-07-30 17:15:09.497845.pt'
'trained_scriptGenModel2019-07-30 17:16:10.999422.pt'
'trained_scriptGenModel2019-07-30 17:16:37.995327.pt'
'trained_scriptGenModel2019-07-30 17:17:48.526540.pt'
'trained_scriptGenModel2019-07-30 17:17:50.015998.pt'


In [0]:
with open('/content/drive/My Drive/Text Generator/place2.txt', 'r') as f:
  text = f.read()

In [6]:
text[:100]

'It was late and every one had left the cafe except an old man \nwho sat in the shadow the leaves of t'

In [0]:
# encode the text and map each character to an integer and vice versa

# we create two dictionaries:
# 1. int2char, which maps integers to characters
# 2. char2int, which maps characters to unique integers
chars = tuple(set(text))
int2char = dict(enumerate(chars))
char2int = {ch: ii for ii, ch in int2char.items()}

# encode the text
encoded = np.array([char2int[ch] for ch in text])

In [8]:
encoded[:100]

array([48, 43, 38, 76, 10, 66, 38, 49, 10, 43, 11, 38, 10, 50, 77, 38, 11,
       55, 11, 19, 59, 38, 31, 50, 11, 38, 12, 10, 77, 38, 49, 11, 75, 43,
       38, 43, 12, 11, 38, 53, 10, 75, 11, 38, 11, 57, 53, 11, 71, 43, 38,
       10, 50, 38, 31, 49, 77, 38, 22, 10, 50, 38, 72, 76, 12, 31, 38, 66,
       10, 43, 38, 23, 50, 38, 43, 12, 11, 38, 66, 12, 10, 77, 31, 76, 38,
       43, 12, 11, 38, 49, 11, 10, 55, 11, 66, 38, 31, 75, 38, 43])

In [0]:
def one_hot_encode(arr, n_labels):
    
    # Initialize the the encoded array
    one_hot = np.zeros((arr.size, n_labels), dtype=np.float32)
    
    # Fill the appropriate elements with ones
    one_hot[np.arange(one_hot.shape[0]), arr.flatten()] = 1.
    
    # Finally reshape it to get back to the original array
    one_hot = one_hot.reshape((*arr.shape, n_labels))
    
    return one_hot

In [12]:
# check that the function works as expected
test_seq = np.array([[3, 5, 1]])
one_hot = one_hot_encode(test_seq, 8)

print(one_hot)

[[[0. 0. 0. 1. 0. 0. 0. 0.]
  [0. 0. 0. 0. 0. 1. 0. 0.]
  [0. 1. 0. 0. 0. 0. 0. 0.]]]


In [0]:

def get_batches(arr, batch_size, seq_length):
    '''Create a generator that returns batches of size
       batch_size x seq_length from arr.
       
       Arguments
       ---------
       arr: Array you want to make batches from
       batch_size: Batch size, the number of sequences per batch
       seq_length: Number of encoded chars in a sequence
    '''
    
    batch_size_total = batch_size * seq_length
    # total number of batches we can make
    n_batches = len(arr)//batch_size_total
    
    # Keep only enough characters to make full batches
    arr = arr[:n_batches * batch_size_total]
    # Reshape into batch_size rows
    arr = arr.reshape((batch_size, -1))
    
    # iterate through the array, one sequence at a time
    for n in range(0, arr.shape[1], seq_length):
        # The features
        x = arr[:, n:n+seq_length]
        # The targets, shifted by one
        y = np.zeros_like(x)
        try:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, n+seq_length]
        except IndexError:
            y[:, :-1], y[:, -1] = x[:, 1:], arr[:, 0]
        yield x, y

In [0]:
batches = get_batches(encoded, 8, 50)
x, y = next(batches)

In [16]:
# printing out the first 10 items in a sequence
print('x\n', x[:10, :10])
print('\ny\n', y[:10, :10])

x
 [[48 43 38 76 10 66 38 49 10 43]
 [ 6 62 12 43 38 43 12 11 38 27]
 [38 76 10 66 38 12 31 43 38 10]
 [12 11 38 77 31 31 19 76 10 59]
 [38 43 12 11 38 66  6 50 38 76]
 [27 11 38 62 31 50 11 38 10 49]
 [38 43 31 38 66 43 10 50 77 38]
 [43 38 27 10 53 52 38 31 75 38]]

y
 [[43 38 76 10 66 38 49 10 43 11]
 [62 12 43 38 43 12 11 38 27 49]
 [76 10 66 38 12 31 43 38 10 49]
 [11 38 77 31 31 19 76 10 59 38]
 [43 12 11 38 66  6 50 38 76 23]
 [11 38 62 31 50 11 38 10 49 49]
 [43 31 38 66 43 10 50 77 38 23]
 [38 27 10 53 52 38 31 75 38 43]]


In [0]:
#self.lstm = nn.LSTM(input_size, n_hidden, n_layers,  dropout=drop_prob, batch_first=True)

In [0]:
#self.init_hidden()

In [17]:
# check if GPU is available
train_on_gpu = torch.cuda.is_available()
if(train_on_gpu):
    print('Training on GPU!')
else: 
    print('No GPU available, training on CPU; consider making n_epochs very small.')

Training on GPU!


In [0]:
class CharRNN(nn.Module):
    
    def __init__(self, tokens, n_hidden=256, n_layers=2,
                               drop_prob=0.5, lr=0.001):
        super().__init__()
        self.drop_prob = drop_prob
        self.n_layers = n_layers
        self.n_hidden = n_hidden
        self.lr = lr
        
        # creating character dictionaries
        self.chars = tokens
        self.int2char = dict(enumerate(self.chars))
        self.char2int = {ch: ii for ii, ch in self.int2char.items()}
        
        ## TODO: define the LSTM
        self.lstm = nn.LSTM(len(self.chars), n_hidden, n_layers, 
                            dropout=drop_prob, batch_first=True)
        
        ## TODO: define a dropout layer
        self.dropout = nn.Dropout(drop_prob)
        
        ## TODO: define the final, fully-connected output layer
        self.fc = nn.Linear(n_hidden, len(self.chars))
      
    
    def forward(self, x, hidden):
        ''' Forward pass through the network. 
            These inputs are x, and the hidden/cell state `hidden`. '''
                
        ## TODO: Get the outputs and the new hidden state from the lstm
        r_output, hidden = self.lstm(x, hidden)
        
        ## TODO: pass through a dropout layer
        out = self.dropout(r_output)
        
        # Stack up LSTM outputs using view
        # you may need to use contiguous to reshape the output
        out = out.contiguous().view(-1, self.n_hidden)

        ## TODO: put x through the fully-connected layer
        out = self.fc(out)
        
        # return the final output and the hidden state
        return out, hidden
    
    
    def init_hidden(self, batch_size):
        ''' Initializes hidden state '''
        # Create two new tensors with sizes n_layers x batch_size x n_hidden,
        # initialized to zero, for hidden state and cell state of LSTM
        weight = next(self.parameters()).data
        
        if (train_on_gpu):
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda(),
                  weight.new(self.n_layers, batch_size, self.n_hidden).zero_().cuda())
        else:
            hidden = (weight.new(self.n_layers, batch_size, self.n_hidden).zero_(),
                      weight.new(self.n_layers, batch_size, self.n_hidden).zero_())
        
        return hidden



        

In [0]:
def train(net, data, epochs=10, batch_size=10, seq_length=50, lr=0.001, clip=5, val_frac=0.1, print_every=10):
    ''' Training a network 
    
        Arguments
        ---------
        
        net: CharRNN network
        data: text data to train the network
        epochs: Number of epochs to train
        batch_size: Number of mini-sequences per mini-batch, aka batch size
        seq_length: Number of character steps per mini-batch
        lr: learning rate
        clip: gradient clipping
        val_frac: Fraction of data to hold out for validation
        print_every: Number of steps for printing training and validation loss
    
    '''
    net.train()
    
    opt = torch.optim.Adam(net.parameters(), lr=lr)
    criterion = nn.CrossEntropyLoss()
    
    # create training and validation data
    val_idx = int(len(data)*(1-val_frac))
    data, val_data = data[:val_idx], data[val_idx:]
    




    if(train_on_gpu):
        net.cuda()
    
    counter = 0
    n_chars = len(net.chars)
    for e in range(epochs):
        # initialize hidden state
        h = net.init_hidden(batch_size)
        
        for x, y in get_batches(data, batch_size, seq_length):
            counter += 1
            
            # One-hot encode our data and make them Torch tensors
            x = one_hot_encode(x, n_chars)
            inputs, targets = torch.from_numpy(x), torch.from_numpy(y)
            
            if(train_on_gpu):
                inputs, targets = inputs.cuda(), targets.cuda()

            # Creating new variables for the hidden state, otherwise
            # we'd backprop through the entire training history
            h = tuple([each.data for each in h])

            # zero accumulated gradients
            net.zero_grad()
            
            # get the output from the model
            output, h = net(inputs, h)
            
            # calculate the loss and perform backprop
            loss = criterion(output, targets.view(batch_size*seq_length).long())
            loss.backward()
            # `clip_grad_norm` helps prevent the exploding gradient problem in RNNs / LSTMs.
            nn.utils.clip_grad_norm_(net.parameters(), clip)
            opt.step()
            
            # loss stats
            if counter % print_every == 0:
                # Get validation loss
                val_h = net.init_hidden(batch_size)
                val_losses = []
                net.eval()
                for x, y in get_batches(val_data, batch_size, seq_length):
                    # One-hot encode our data and make them Torch tensors
                    x = one_hot_encode(x, n_chars)
                    x, y = torch.from_numpy(x), torch.from_numpy(y)
                    
                    # Creating new variables for the hidden state, otherwise
                    # we'd backprop through the entire training history
                    val_h = tuple([each.data for each in val_h])
                    
                    inputs, targets = x, y
                    if(train_on_gpu):
                        inputs, targets = inputs.cuda(), targets.cuda()

                    output, val_h = net(inputs, val_h)
                    val_loss = criterion(output, targets.view(batch_size*seq_length).long())
                
                    val_losses.append(val_loss.item())
                
                net.train() # reset to train mode after iterationg through validation data
                
                print("Epoch: {}/{}...".format(e+1, epochs),
                      "Step: {}...".format(counter),
                      "Loss: {:.4f}...".format(loss.item()),
                      "Val Loss: {:.4f}".format(np.mean(val_losses)))

           

In [22]:

# define and print the net
n_hidden=512
n_layers=2

net = CharRNN(chars, n_hidden, n_layers)
print(net)

CharRNN(
  (lstm): LSTM(78, 512, num_layers=2, batch_first=True, dropout=0.5)
  (dropout): Dropout(p=0.5)
  (fc): Linear(in_features=512, out_features=78, bias=True)
)


In [23]:
batch_size = 128
seq_length = 100
n_epochs = 20 # start smaller if you are just testing initial behavior

# train the model
train(net, encoded, epochs=n_epochs, batch_size=batch_size, seq_length=seq_length, lr=0.001, print_every=10)

/usr/local/lib/python3.6/dist-packages/numpy/core/fromnumeric.py:3118: RuntimeWarning: Mean of empty slice.
  out=out, **kwargs)
/usr/local/lib/python3.6/dist-packages/numpy/core/_methods.py:85: RuntimeWarning: invalid value encountered in double_scalars
  ret = ret.dtype.type(ret / rcount)


Epoch: 3/20... Step: 10... Loss: 3.2210... Val Loss: nan
Epoch: 5/20... Step: 20... Loss: 3.1389... Val Loss: nan
Epoch: 8/20... Step: 30... Loss: 3.1285... Val Loss: nan
Epoch: 10/20... Step: 40... Loss: 3.1041... Val Loss: nan
Epoch: 13/20... Step: 50... Loss: 3.1117... Val Loss: nan
Epoch: 15/20... Step: 60... Loss: 3.0926... Val Loss: nan
Epoch: 18/20... Step: 70... Loss: 3.0989... Val Loss: nan
Epoch: 20/20... Step: 80... Loss: 3.0758... Val Loss: nan


In [0]:
# change the name, for saving multiple files
model_name = 'rnn_20_epoch.net'

checkpoint = {'n_hidden': net.n_hidden,
              'n_layers': net.n_layers,
              'state_dict': net.state_dict(),
              'tokens': net.chars}

with open(model_name, 'wb') as f:
    torch.save(checkpoint, f)

In [0]:
def predict(net, char, h=None, top_k=None):
        ''' Given a character, predict the next character.
            Returns the predicted character and the hidden state.
        '''
        
        # tensor inputs
        x = np.array([[net.char2int[char]]])
        x = one_hot_encode(x, len(net.chars))
        inputs = torch.from_numpy(x)
        
        if(train_on_gpu):
            inputs = inputs.cuda()
        
        # detach hidden state from history
        h = tuple([each.data for each in h])
        # get the output of the model
        out, h = net(inputs, h)

        # get the character probabilities
        p = F.softmax(out, dim=1).data
        if(train_on_gpu):
            p = p.cpu() # move to cpu
        
        # get top characters
        if top_k is None:
            top_ch = np.arange(len(net.chars))
        else:
            p, top_ch = p.topk(top_k)
            top_ch = top_ch.numpy().squeeze()
        
        # select the likely next character with some element of randomness
        p = p.numpy().squeeze()
        char = np.random.choice(top_ch, p=p/p.sum())
        
        # return the encoded value of the predicted char and the hidden state
        return net.int2char[char], h

In [0]:
def sample(net, size, prime='The', top_k=None):
        
    if(train_on_gpu):
        net.cuda()
    else:
        net.cpu()
    
    net.eval() # eval mode
    
    # First off, run through the prime characters
    chars = [ch for ch in prime]
    h = net.init_hidden(1)
    for ch in prime:
        char, h = predict(net, ch, h, top_k=top_k)

    chars.append(char)
    
    # Now pass in the previous character and get a new one
    for ii in range(size):
        char, h = predict(net, chars[-1], h, top_k=top_k)
        chars.append(char)

    return ''.join(chars)

In [28]:

print(sample(net, 1000, prime='man', top_k=5))

mantettt eeatte ee otoe t ao oaatee   eo e eoo    eeato tee attoeetaa o t o oe etottta   tteatoo t a toe  ee   tt o      ea t ett ao  ee   o a      a otoo   a   tee  ete etatt a eoae taeteo  t aaa te to tto  t e oaea o aa  a o aeo  e tot t o a   eeaet  t   e  ao  otao t a otee  tat ae  aa  ae oae e    t  t  ae a e  a  tao tt oao et    oe   oeo teet e  tto ae et e  oa eott  atoee e at a oooteaoe tttaea ee e    eot t teoe  t  ot eoetao oe a ee eaa  e t t a ae  tao tott oeotte ea tae  t ee  ot a  te  oet  a  ttatote  eet e aaa  e t aet eao t e a ae  e    ot o      t att  aatttoa e e  te   eat  tte eae   ee  a  e  a  oaeat oa  o  e ea oeo  te e eeaoatoae e  ato     o   oaoa  ottaooee  tt e tae a   tot  e e eo et  e  ott ao  o otatooe oe  t  o eee o aatoeeaeat eteooe aot aootea a  a  to oee teeaea e taeo tte  oa ot ea a   ot aoeta e   to e te t   o   e  ta  etto  aoe oat aoe aeto tet toa a ateeee  et oata  a te  o ooeo o otete o    aea eeet e  ao  eooo t  et ato a  ett t o eoa  eee  e t   t

In [34]:
# Here we have loaded in a model that trained over 20 epochs `rnn_20_epoch.net`
with open('rnn_20_epoch.net', 'rb') as f:
    checkpoint = torch.load(f)
    
loaded = CharRNN(checkpoint['tokens'], n_hidden=checkpoint['n_hidden'], n_layers=checkpoint['n_layers'])
loaded.load_state_dict(checkpoint['state_dict'])

IncompatibleKeys(missing_keys=[], unexpected_keys=[])

In [35]:
# Sample using a loaded model
print(sample(loaded, 2000, top_k=5, prime="clean place"))

clean placet  eetaea ee  e ee aot  t t eo e  ae  e  te tet o tattt  att e etoeta a eaa  ttt  oaa o oo a ae  toaa e a  ta taa e o  eoooto taotaaaataeotteaeaoteee  aaat   e ea e  ot      o eteaoeaa att etootae  otoetaa oe   aaae  e e aottee t eae et e et o et oe  aoo oaat  tot eee  a   e oe eeo    eaooa e otta    ta   e  e aeo toeeao aoatt   oet eea  eoee  aatooa e  oaet   t o e aeao o ao  to  eaat  e e tee teea  t teao to a toa  oottoo at e  ee      eoteoe a et  t   aoe o t  e   eo    oe a o ooeote  toeoa  o oe te o  e aeet e oteottete  aee  aeot ae etee to    e e  eetet  o  tete tott a t e oot toaaa ee  oea  ea to toe e a otaoe  e aa e ta aee ot eeeae    eetat  tooe  e ot  tta o  oe o   o ea   eaa  o aaoeeattet a a e att taee  ae  tat te    t a  ao t    t e eate ae t  et t     teao aaeao t to o  e eoo  aa eteee  o a eae eoa eet   eeoaoteta tt ae ee eoettto  o ett    ao tte  toatao  a e e  toaoo  ooa eeat ot ott ette  tetoo oe   ott e o  ee oataa   e   eta tte ae   o  tae oeoto tee tta 